<a href="https://colab.research.google.com/github/mantoan-thi/-Python-for-investments-/blob/main/Stock_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas-datareader

In [14]:
import pandas as pd
from pandas_datareader import data
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import time
import numpy as np
# bibliotecas para trabalhar offline com a biblioteca
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

In [15]:
import plotly.io as pio
pio.renderers
pio.renderers.default = 'colab'
pio.templates.default = "plotly_dark"

In [3]:
tickers = ['RRRP3.SA',	'CSAN3.SA',	'IGTI11.SA',	'PETZ3.SA','ALPA4.SA','CPFE3.SA','GNDI3.SA','POSI3.SA',
          'ABEV3.SA',	'CMIN3.SA',	'IRBR3.SA',	'QUAL3.SA','AMER3.SA',	'CVCB3.SA',	'ITSA4.SA',	'RADL3.SA',
          'ASAI3.SA',	'CYRE3.SA',	'ITUB4.SA',	'RDOR3.SA','AZUL4.SA',	'DXCO3.SA',	'JBSS3.SA',	'RAIL3.SA',
          'B3SA3.SA',	'ECOR3.SA',	'JHSF3.SA',	'SBSP3.SA','BIDI11.SA',	'ELET3.SA',	'KLBN11.SA',	'SANB11.SA',
          'BPAN4.SA',	'ELET6.SA',	'RENT3.SA',	'CSNA3.SA','BBSE3.SA',	'EMBR3.SA',	'LCAM3.SA',	'SULA11.SA',
          'BRML3.SA',	'ENBR3.SA',	'LWSA3.SA',	'SUZB3.SA','BBDC3.SA',	'ENGI11.SA',	'LAME4.SA',	'TAEE11.SA',
          'BBDC4.SA',	'ENEV3.SA',	'LREN3.SA',	'VIVT3.SA','BRAP4.SA',	'EGIE3.SA',	'MGLU3.SA',	'TIMS3.SA',
          'BBAS3.SA',	'EQTL3.SA',	'MRFG3.SA',	'TOTS3.SA','BRKM5.SA',	'EZTC3.SA',	'CASH3.SA',	'UGPA3.SA',
          'BRFS3.SA',	'FLRY3.SA',	'BEEF3.SA',	'USIM5.SA','BPAC11.SA',	'GGBR4.SA',	'MRVE3.SA',	'VALE3.SA',
          'CRFB3.SA',	'GOAU4.SA',	'MULT3.SA',	'VIIA3.SA','CCRO3.SA',	'GOLL4.SA',	'PCAR3.SA',	'VBBR3.SA',
          'CMIG4.SA',	'NTCO3.SA',	'PETR3.SA',	'WEGE3.SA','CIEL3.SA',	'SOMA3.SA',	'PETR4.SA',	'YDUQ3.SA',
          'COGN3.SA',	'HAPV3.SA',	'PRIO3.SA',	'HYPE3.SA','CPLE6.SA']

In [24]:
def coletar(tickers):
    start_date = '2015-01-01'
    end_date = '2016-12-31'
    dados = data.DataReader(tickers, 'yahoo', start_date)['Adj Close']
    dataset = pd.DataFrame(dados)
    dataset.reset_index(inplace=True)
    dataset.rename(columns={'Date': 'ds','Adj Close':'y'},inplace =True)
    return dataset

def previsao(dados):
    #prior_scale=0.2
    m = Prophet()
    m.fit(dados)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    #forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    return forecast['yhat'].iloc[-31:]

In [25]:
def gerar_analise():
  lista = []
  contador = 0
  print('Carteira Teórica do IBovespa\n')
  print('Quantidade de ativos: ',len(tickers),'\n')
  for ativo in tickers:
      print('Analisando: ',ativo,' // ',contador,' de ',len(tickers),' {:.2%}'.format(contador/len(tickers)))
      dataset = previsao(coletar(ativo))
      var = dataset[-1:].values[0]/dataset[:1].values[0]-1
      lista.append([ativo,var])
      contador +=1
      time.sleep(1)
  new_lista = pd.DataFrame(lista,columns=['Ativo','Percentual'])
  Ativo_compra = new_lista[new_lista['Percentual'] > 0]
  Ativo_venda = new_lista[new_lista['Percentual'] < 0]
  Ativo_compra.sort_values(by=['Percentual'],ascending=False,inplace=True)
  Ativo_venda.sort_values(by=['Percentual'],inplace=True)
  Ativo_compra['Percentual'] = Ativo_compra['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  Ativo_venda['Percentual'] = Ativo_venda['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  return Ativo_compra,Ativo_venda

In [ ]:
Ativo_compra, Ativo_venda = gerar_analise()

In [27]:
print('Compra:',len(Ativo_compra),'// Venda: ',len(Ativo_venda))

Compra: 72 // Venda:  21


In [39]:
Ativo_compra.head(30)

,Ativo,Percentual
78,PCAR3.SA,34.51%
10,IRBR3.SA,33.72%
84,CIEL3.SA,16.33%
20,AZUL4.SA,13.39%
40,BRML3.SA,7.23%
68,BPAC11.SA,6.65%
90,PRIO3.SA,6.24%
59,TOTS3.SA,6.13%
32,BPAN4.SA,6.02%
6,GNDI3.SA,6.00%


In [29]:
Ativo_venda

,Ativo,Percentual
62,CASH3.SA,-68.33%
46,LAME4.SA,-32.62%
42,LWSA3.SA,-19.35%
19,RDOR3.SA,-19.09%
88,COGN3.SA,-16.82%
2,IGTI11.SA,-15.53%
16,ASAI3.SA,-15.01%
3,PETZ3.SA,-14.52%
85,SOMA3.SA,-10.51%
81,NTCO3.SA,-8.61%


In [37]:
nome_ativo = Ativo_compra['Ativo'].iloc[:1].values[0]
new_dataset = coletar(nome_ativo)

In [38]:
new_dataset['Media_21'] = new_dataset['y'].rolling(window=21).mean()
new_dataset['Media_50'] = new_dataset['y'].rolling(window=50).mean()

trace0 = go.Scatter(x=new_dataset.ds, y=new_dataset['y'], name=nome_ativo)
trace1 = go.Scatter(x=new_dataset.ds, y=new_dataset['Media_21'], name='21-day MA')
trace2 = go.Scatter(x=new_dataset.ds, y=new_dataset['Media_50'], name='50-day MA')
fig1 = go.Figure([trace0, trace1, trace2])
fig1.update_layout(
    title={
        'text': 'ticker ('+ nome_ativo +') Stock Price',
        'x':0.5,
        'xanchor': 'center'})
fig1.show()